# Exemplo de Agente de Reserva de Hotel e Voos

Esta solução irá ajudá-lo a reservar bilhetes de avião e hotel. O cenário é uma viagem de Londres Heathrow LHR a 20 de fevereiro de 2024 para Nova Iorque JFK, retornando a 27 de fevereiro de 2025, voando em classe económica apenas com a British Airways. Quero ficar num hotel Hilton em Nova Iorque, por favor forneça os custos do voo e do hotel.


# Inicializar o Serviço Azure AI Agent e obter informações de configuração a partir de **.env**

### **.env**

Crie um ficheiro .env

O ficheiro **.env** contém a string de conexão do Serviço Azure AI Agent, o modelo utilizado pelo AOAI e o serviço correspondente da API de Pesquisa do Google, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nome da Implementação do Modelo do Serviço Azure AI Agent"

[**NOTE**] Será necessário um modelo com um limite de 100.000 tokens por minuto e um limite de 600 solicitações por minuto.

  Pode obter o modelo no Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "String de Conexão do Projeto do Serviço Azure AI Agent"

  Pode obter a string de conexão do projeto na visão geral do projeto no ecrã do Portal AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "A sua chave API de Pesquisa SERPAPI"
- **SERPAPI_SEARCH_ENDPOINT** = "O seu Endpoint de Pesquisa SERPAPI"

Para obter o Nome da Implementação do Modelo e a String de Conexão do Projeto do Serviço Azure AI Agent, é necessário criar o Serviço Azure AI Agent. Recomenda-se utilizar [este modelo](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) para criá-lo diretamente (***Nota:*** O Serviço Azure AI Agent está atualmente configurado numa região limitada. Recomenda-se consultar [este link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) para definir a região).

O agente precisa de acesso ao SERPAPI. Recomenda-se registar-se utilizando [este link](https://serpapi.com/searches). Após o registo, pode obter uma chave API única e um ENDPOINT.


# Configuração

Para executar este notebook, certifique-se de que instalou as bibliotecas necessárias executando `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

A sua versão do Semantic Kernel deve ser pelo menos 1.27.2.


Carregue as definições e recursos do seu ficheiro .env, por favor, certifique-se de que adicionou as suas chaves e definições e criou um ficheiro .env local.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Iniciar sessão no Azure

Agora precisa de iniciar sessão no Azure. Abra um terminal e execute o seguinte comando:

```bash
az login
```

Este comando irá pedir-lhe para introduzir as suas credenciais do Azure, permitindo que o serviço Azure AI Agent funcione corretamente.


# Explicação:
Esta é uma variável que armazena a chave da API para aceder a um serviço de API de SERP (Página de Resultados de Motor de Busca). Uma chave de API é um identificador único usado para autenticar pedidos associados à sua conta.

Objetivo: O objetivo desta linha é armazenar a chave da API numa variável para que possa ser usada para autenticar pedidos ao serviço de API de SERP. A chave da API é necessária para aceder ao serviço e realizar pesquisas.

Como Obter uma Chave de API de SERP: Para obter uma chave de API de SERP, siga estes passos gerais em https://serpapi.com (os passos exatos podem variar dependendo do serviço de API de SERP específico que está a utilizar):

Escolha um Serviço de API de SERP: Existem vários serviços de API de SERP disponíveis, como SerpAPI, Google Custom Search JSON API, entre outros. Escolha o que melhor se adapta às suas necessidades.

Registe-se para uma Conta: Aceda ao site do serviço de API de SERP escolhido e registe-se para uma conta. Pode ser necessário fornecer algumas informações básicas e verificar o seu endereço de email.

Crie uma Chave de API: Após o registo, inicie sessão na sua conta e navegue até à secção de API ou painel de controlo. Procure uma opção para criar ou gerar uma nova chave de API.  
Copie a chave da API para o seu ficheiro .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explicação:
BASE_URL: Esta é uma variável que armazena o URL base para o endpoint da API SERP. O nome da variável BASE_URL é uma convenção usada para indicar que este URL é o ponto de partida para fazer pedidos à API.

'https://serpapi.com/search':

Este é o valor real do URL atribuído à variável BASE_URL. Representa o endpoint para realizar consultas de pesquisa utilizando a API SERP.

# Objetivo:
O objetivo desta linha é definir uma constante que contém o URL base para a API SERP. Este URL será usado como ponto de partida para construir pedidos à API e realizar operações de pesquisa.

# Utilização:
Ao definir o URL base numa variável, pode reutilizá-lo facilmente ao longo do seu código sempre que precisar de fazer pedidos à API SERP. Isto torna o seu código mais fácil de manter e reduz o risco de erros ao evitar a codificação manual do URL em vários locais. O exemplo atual é https://serpapi.com/search?engine=bing, que utiliza a API de pesquisa Bing. Diferentes APIs podem ser selecionadas em https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Explicação:

Este é o local onde o código do seu plugin está armazenado.

Definição da Classe: `class BookingPlugin`: Define uma classe chamada BookingPlugin que contém métodos para reservar hotéis e voos.

Método de Reserva de Hotel:

- `@kernel_function(description="booking hotel")`: Um decorador que descreve a função como uma função kernel para reserva de hotéis.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Define um método para reservar hotéis com parâmetros anotados e tipo de retorno.

O método constrói um dicionário de parâmetros para a solicitação de reserva de hotel e envia uma solicitação GET para a API SERP. Verifica o status da resposta e retorna as propriedades do hotel se for bem-sucedido, ou None se a solicitação falhar.

Método de Reserva de Voos:

- `@kernel_function(description="booking flight")`: Um decorador que descreve a função como uma função kernel para reserva de voos.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Define um método para reservar voos com parâmetros anotados e tipo de retorno.

O método constrói dicionários de parâmetros para as solicitações de voos de ida e volta e envia solicitações GET para a API SERP. Verifica o status da resposta e adiciona as informações do voo à string de resultados se for bem-sucedido, ou imprime uma mensagem de erro se a solicitação falhar. O método retorna a string de resultados contendo as informações do voo.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Explicação:
Declarações de Importação: Importar os módulos necessários para credenciais do Azure, agente de IA, conteúdo de mensagens de chat, função de autor e decorador de função do kernel.

Gestor de Contexto Assíncrono: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Isto configura um gestor de contexto assíncrono para lidar com as credenciais do Azure e criar um cliente de agente de IA.

Nome e Instruções do Agente:
- `AGENT_NAME = "BookingAgent"`: Define o nome do agente.
- `AGENT_INSTRUCTIONS = """..."""`: Fornece instruções detalhadas para o agente sobre como lidar com pedidos de reserva.

Criar Definição do Agente: `agent_definition = await client.agents.create_agent(...)`: Cria uma definição de agente com o modelo, nome e instruções especificados.

Criar Agente AzureAI: `agent = AzureAIAgent(...)`: Cria um agente AzureAI utilizando o cliente, a definição do agente e o plugin definido.

Criar Thread: `thread: AzureAIAgentThread | None = None`: Cria uma thread para o agente. Não é necessário criar uma thread primeiro - se o valor `None` for fornecido, uma nova thread será criada durante a primeira invocação e retornada como parte da resposta.

Entradas do Utilizador: `user_inputs = ["..."]`: Define uma lista de entradas do utilizador para o agente processar.

No bloco finally, apague a thread e o agente para libertar os recursos.


# Autenticação

A classe `DefaultAzureCredential` faz parte do Azure SDK para Python. Ela oferece uma forma padrão de autenticação com os serviços Azure. Tenta autenticar utilizando vários métodos numa ordem específica, como variáveis de ambiente, identidade gerida e credenciais do Azure CLI.

Operações Assíncronas: O módulo aio indica que a classe DefaultAzureCredential suporta operações assíncronas. Isto significa que pode ser utilizada com asyncio para realizar pedidos de autenticação não bloqueantes.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante ter em conta que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
